In [2]:
from os import listdir
from os.path import isfile, join
import pandas as pd, numpy as np
from scipy import stats
onlyfiles = [f for f in listdir('WP') if isfile(join('WP', f))]
tms = pd.Series(onlyfiles)
tms

0                                CC Price Jan-Aug.xlsx
1          Final 1 โครงสร้างไม้เดือนกันยายน 63 r4.xlsx
2              Finalโครงสร้างไม้เดือนเมษายน 63 V4.xlsx
3                  โครงสร้างไม้เดือนกรกฎาคม 63 V6.xlsx
4               โครงสร้างไม้เดือนกุมภาพันธ์ 63v 5.xlsx
5                  โครงสร้างไม้เดือนพฤษภาคม 63 V2.xlsx
6                    โครงสร้างไม้เดือนมกราคม63 V4.xlsx
7    โครงสร้างไม้เดือนมิถุนายน 63 v4 1(ปรับunit use...
8    โครงสร้างไม้เดือนมีนาคม 63 v4ปรับระหว่างเดือน....
9                     โครงสร้างไม้เดือนสิงหาคม 63.xlsx
dtype: object

In [5]:
fn = tms[tms.str.contains('มกราคม')].values[0]

In [6]:
pd.read_excel('WP/'+fn, sheet_name='โครงสร้างไม้', index_col=0, header=2).columns

Index([                                   'เขต',
                                      'Cluster',
                                      'รายชื่อ',
                             'Storage Location',
                                 'Wood Channel',
                                  'Vendor Type',
                                    'Wood Type',
                                     'แหล่งไม้',
                                      'คู่แข่ง',
                                  'ราคาคู่แข่ง',
                             'ปริมาณส่งคู่แข่ง',
                            'ปริมาณไม้ทั้งหมด ',
                              'SFT ปริมาณจัดหา',
                          'ปริมาณ จัดส่ง \n WS',
                            'ปริมาณจัดส่ง \nBP',
                           'ปริมาณจัดส่ง \nKPP',
                           'ปริมาณจัดส่ง \nCHP',
                           'ปริมาณจัดส่ง \nPHY',
                   'ปริมาณจัดส่ง\nTotal volume',
                          'รวมต้นทุนไม้ทั้งหมด',
                    

In [7]:
def CPF(x):
    cpi = [i for i in x if i>0]
    if len(cpi) > 1:
        zs  = stats.zscore(cpi); fzs=[]
        for i in range(len(cpi)):
            if np.abs(zs[i]) < 1.4:
                fzs.append(cpi[i])
        return fzs
    else:
        return cpi

In [8]:
SADA = {} #SADA, SAD, CPM, CPSD = {}, {}, {}, {}
for tm_ in tm.keys(): # ['January']: #
    fn = tms[tms.str.contains(tm[tm_])].values[0]
    ex = pd.read_excel('WP/'+fn, sheet_name='โครงสร้างไม้', index_col=0, header=2)
    
    # Get only rows with Sloc
    N = np.array(ex['Storage Location'].str.contains('N').fillna(False))
    W = np.array(ex['Storage Location'].str.contains('W').fillna(False))
    NW= N+W
    
    # Clean Competitor Price
    cp = ex['ราคาคู่แข่ง'].apply(lambda x: str(x)[:5]).str.replace('-','').str.replace(',','')
    cpi = cp.astype(float)
    #cpi = cpi.fillna(0)
    ex['CP'] = cpi
    
    # Group Agents and CP mby Sloc
    Sloc = ex[NW].groupby('Storage Location')['รายชื่อ'].unique().index
    Agent= ex[NW].groupby('Storage Location')['รายชื่อ'].unique().values
    nCP  = ex[NW].groupby('Storage Location')['คู่แข่ง'].unique().values
    CP   = ex[NW].groupby('Storage Location')['CP'].unique().apply(CPF)
    QC   = ex[NW].groupby('Storage Location')['ปริมาณส่งคู่แข่ง'].sum()
    QT   = ex[NW].groupby('Storage Location')['ปริมาณไม้ทั้งหมด '].sum()
    QS   = ex[NW].groupby('Storage Location')['SFT ปริมาณจัดหา'].sum()
    
    SA = pd.DataFrame({'Agent': Agent, 'CP': CP, 'nCP':nCP, 'QC':QC, 'QT':QT, 'QS':QS}, index=Sloc)
    SA['nAgent'] = SA['Agent'].apply(len)
    SA['nCP']    = SA['nCP'].apply(len)
    SA['CPM']    = SA['CP'].apply(np.mean)
    SA['CPSD']   = SA['CP'].apply(np.std)
    SA['Year']   = '2020'
    SA['Month']  = tm_
    
    # Save and Print Information
    #SAD[tm_] = SA['nAgent']#.sort_values(by='nAgent', ascending=False)
    #CPM[tm_] = SA['CPM']#.sort_values(by='nAgent', ascending=False)
    #CPSD[tm_]= SA['CPSD']#.sort_values(by='nAgent', ascending=False)
    SADA[tm_] = SA
    print(tm_, SA['nAgent'].sum(), SA['nAgent'].mean(), SA['nCP'].mean(), SA['CPM'].mean(), SA['CPSD'].mean())

1 212 5.170731707317073 1.3902439024390243 1199.9593495934957 50.885890582815016
2 206 5.024390243902439 1.2439024390243902 1188.1707317073171 34.51219512195122
3 239 5.558139534883721 1.2790697674418605 1190.2325581395348 35.348837209302324
4 258 6.142857142857143 1.2857142857142858 1166.4285714285713 36.19047619047619
5 309 7.536585365853658 1.3170731707317074 1145.6097560975609 29.969270213173616
6 307 7.487804878048781 1.2682926829268293 1135.2439024390244 20.853658536585368
7 301 7.341463414634147 1.2682926829268293 1155.9756097560976 31.829268292682926
8 247 6.024390243902439 1.2439024390243902 1170.4065040650407 40.05758430356664
9 227 5.536585365853658 1.2439024390243902 1171.7601626016262 38.24051113283494


In [ ]:
SADA['1']

In [ ]:
SADA['1']['CPM'].mean()

In [ ]:
import datetime as dt
dt.datetime.now().year+1

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
X, y = make_regression(n_features=4, n_informative=2, random_state=0, shuffle=False)

In [ ]:
pd.DataFrame({'a':[1,2]}).append(pd.Series({'b':3}), ignore_index=True)